# Pruning with our Automatic Structured Pruning framework
Welcome to an end-to-end example for automatic structured weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for CIFAR10 from scratch.
* Fine tune the model by applying the different methods of the pruning Framework and see the accuracy.

If you want to execute this notebook in Google Colab, uncomment the code below.

In [1]:
import sys

if 'google.colab' in sys.modules:
    !git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
    !echo $CWD
    sys.path.append("Automatic-Structured-Pruning/src")
else:
    sys.path.append("../src")
    
import pruning

In [1]:
import asp as pruning

In [8]:
pruning.load_model_param(model)

(array(['Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D',
        'MaxPooling2D', 'Flatten', 'Dense', 'Dropout', 'Dense', 'Dropout',
        'Dense'], dtype='<U12'),
 array([list([array([[[[-6.52834401e-02, -3.56518440e-02, -7.95705840e-02,
            3.38718705e-02,  8.85325447e-02,  5.46130128e-02,
            2.77920328e-02, -1.12264410e-01, -1.01500168e-01,
           -6.32410794e-02,  7.42117837e-02, -1.39368460e-01,
            9.42386314e-02, -1.47341162e-01, -6.25845417e-02,
           -8.19835663e-02, -1.31177917e-01,  5.23018930e-03,
           -7.08303526e-02, -1.15423594e-02, -8.97122249e-02,
            4.59017865e-02, -5.51356934e-02,  2.55234614e-02,
           -1.17180474e-01, -1.21207207e-01, -1.15345851e-01,
           -3.34145129e-02,  1.33200079e-01, -6.27124682e-02,
            4.92826328e-02,  1.61471859e-01],
          [-5.98537996e-02, -5.83897941e-02, -2.94794887e-02,
           -9.23425555e-02,  5.35296425e-02, -1.00822181e-01,
           -6.34120

## Train a model for CIFAR10 without pruning
Download and prepare the CIFAR10 dataset.
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

## Compile and train the model

In [3]:
comp = {
"optimizer":'adam',
"loss": tf.keras.losses.SparseCategoricalCrossentropy(),
"metrics": ['accuracy']
}

model.compile(**comp)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]

model.fit(train_images, train_labels, validation_split=0.2, epochs=1, batch_size=128, callbacks=callbacks)

model_test_loss, model_test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Model accuracy after Training: {model_test_acc*100:.2f}%")

313/313 [==============================] - 7s 8ms/step - loss: 2.0090 - accuracy: 0.2347 - val_loss: 1.6266 - val_accuracy: 0.4130
313/313 - 1s - loss: 1.6240 - accuracy: 0.4134
Model accuracy after Training: 41.34%


# Prune a model
You will apply pruning to the whole model and see this in the model summary. In this example, you prune the model with 50% dense pruning and 40% filter pruning. Therefore, the `factor_pruning` method of the framework is called.

In [4]:
import asp as pruning

In [5]:
dir(pruning)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'accuracy_pruning',
 'build_pruned_model',
 'factor_pruning',
 'get_last_layer_with_params',
 'load_model_param',
 'model_pruning',
 'pruning',
 'pruning_helper_classes',
 'pruning_helper_functions',
 'pruning_helper_functions_conv',
 'pruning_helper_functions_dense',
 'stepwise_accuracy_pruning',
 'stepwise_factor_pruning']

In [6]:
dense_prune_rate=50
conv_prune_rate=40
pruned_model=pruning.factor_pruning(model, dense_prune_rate, conv_prune_rate,'L1')

Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)   

C:\Users\dk100\Anaconda3\envs\asp_2\lib\site-packages\asp\pruning_helper_functions.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(layer_types), np.array(layer_params), \


We see how we get less parameter in the pruned model.

## Compile and re-train the model

In [7]:
pruned_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 20)        560       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 39)        7059      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 39)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 39)          13728     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 39)          0         
_________________________________________________________________
flatten (Flatten)            (None, 156)               0

In [8]:
pruned_model.compile(**comp)

pruned_model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
1250/1250 [==============================] - 7s 5ms/step - loss: 1.7897 - accuracy: 0.3162 - val_loss: 1.5500 - val_accuracy: 0.4394
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.5788 - accuracy: 0.4155 - val_loss: 1.4069 - val_accuracy: 0.4851
Epoch 3/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.4673 - accuracy: 0.4645 - val_loss: 1.2933 - val_accuracy: 0.5295
Epoch 4/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.3853 - accuracy: 0.4981 - val_loss: 1.2477 - val_accuracy: 0.5500
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.3211 - accuracy: 0.5252 - val_loss: 1.2324 - val_accuracy: 0.5644
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.2636 - accuracy: 0.5482 - val_loss: 1.1377 - val_accuracy: 0.5942
Epoch 7/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.2231 - accuracy: 0.5640 - val_loss: 1.1307 - val_accuracy:

Compare both models

In [9]:
model_test_loss, model_test_acc = model.evaluate(test_images,  test_labels, verbose=2)
pruned_model_test_loss, pruned_model_test_acc = pruned_model.evaluate(test_images,  test_labels, verbose=2)

print(f"Model accuracy before pruning: {model_test_acc*100:.2f}%")
print(f"Model accuracy after pruning: {pruned_model_test_acc*100:.2f}%")

313/313 - 1s - loss: 1.6454 - accuracy: 0.3963
313/313 - 1s - loss: 1.0611 - accuracy: 0.6323
Model accuracy before pruning: 39.63%
Model accuracy after pruning: 63.23%


In [10]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {pruned_model.count_params()}")
print(f"Pruned model contains only {(pruned_model.count_params()/model.count_params())*100:.2f}% of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 28480
Pruned model contains only 37.88% of the original number of parameters.


# Prune a model to a maximum accuracy loss

In this case, we only want to have an accuracy loss of 3%. Therefore, the `accuracy_pruning` method of the framework is called. It is started with a pruning factor (for dense and conv) of 5. The weights and filters are removed and the network is re-trained. If the minimum accuracy is reached, the pruning factor is increased and the original model is reduced with the new pruning factor. The model is then re-trained again. This is done until the minimum accuracy is no longer reached.

In [11]:
auto_model = pruning.accuracy_pruning(model, comp, train_images, train_labels, test_images,
                                     test_labels, pruning_acc=None, max_acc_loss=3,
                                     label_one_hot=False)

313/313 [==============================] - 1s 2ms/step - loss: 1.6454 - accuracy: 0.3963
Start model accuracy: 39.63 %
Minimum required model accuracy: 36.63 %
Next pruning factors: 5
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
__________________________________

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 1.6872 - accuracy: 0.3658 - val_loss: 1.4271 - val_accuracy: 0.4883
Next pruning factors: 35
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
___________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8483 - accuracy: 0.2844 - val_loss: 1.6134 - val_accuracy: 0.3908
Next pruning factors: 65
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
___________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 2.0717 - accuracy: 0.1718 - val_loss: 1.9061 - val_accuracy: 0.2141
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 1.9518 - accuracy: 0.2010 - val_loss: 1.8674 - val_accuracy: 0.2555
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 1.9049 - accuracy: 0.2266 - val_loss: 1.8090 - val_accuracy: 0.2703
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8678 - accuracy: 0.2401 - val_loss: 1.7543 - val_accuracy: 0.2987
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8421 - accuracy: 0.2522 - val_loss: 1.7287 - val_accuracy: 0.3191
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8284 - accuracy: 0.2608 - val_loss: 1.7125 - val_accuracy: 0.3170
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8098 - accuracy: 0.2727 - val_loss: 1.6997 - val_accuracy: 0.3312
Epoch 

In [12]:
auto_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 7)         196       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 7)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 13)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 13)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 13)          1534      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 13)          0         
_________________________________________________________________
flatten (Flatten)            (None, 52)                0

Compare both models

In [13]:
model_test_loss, model_test_acc = model.evaluate(test_images,  test_labels, verbose=2)
auto_model_test_loss, auto_model_test_acc = auto_model.evaluate(test_images,  test_labels, verbose=2)

print(f"Model accuracy before pruning: {model_test_acc*100:.2f}%")
print(f"Model accuracy after pruning: {auto_model_test_acc*100:.2f}%")

313/313 - 1s - loss: 1.6454 - accuracy: 0.3963
313/313 - 1s - loss: 1.6556 - accuracy: 0.3670
Model accuracy before pruning: 39.63%
Model accuracy after pruning: 36.70%


In [14]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {auto_model.count_params()}")
print(f"Pruned model contains only {(auto_model.count_params()/model.count_params())*100:.2f}% of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 3429
Pruned model contains only 4.56% of the original number of parameters.


# Prune a model stepwise

The model is not to be pruned all at once but step by step. Therefore, the `stepwise_factor_pruning` method of the framework is called. The pruning factors for dense and conv layers are defined. In addition, the number of steps is defined, how often the model should be reduced. In each step, the model is first reduced and then re-trained. In this way, the model is reduced step by step.

In [15]:
step_factor_model = pruning.stepwise_factor_pruning(model, train_images, train_labels, test_images,
                               test_labels, prun_factor_dense=10, prun_factor_conv=10,
                               num_steps=10, comp=comp)

pruning step: 1/10
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
f

Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 1.3024 - accuracy: 0.5396 - val_loss: 1.1164 - val_accuracy: 0.6103
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 1.0518 - accuracy: 0.6385 - val_loss: 0.9630 - val_accuracy: 0.6623
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.9683 - accuracy: 0.6672 - val_loss: 0.9592 - val_accuracy: 0.6627
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.9250 - accuracy: 0.6859 - val_loss: 0.9406 - val_accuracy: 0.6726
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.8919 - accuracy: 0.6951 - val_loss: 0.9203 - val_accuracy: 0.6791
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.8570 - accuracy: 0.7069 - val_loss: 0.9447 - val_accuracy: 0.6778
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.8231 - accuracy: 0.7214 - val_loss: 0.8771 - val_accuracy: 0.7029
Epoch 

After pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 23)        644       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 23)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 44)        9152      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 44)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 44)          17468     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 44)          0         
_________________________________________________________________
flatten (Flatten)            (None, 176) 

Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 1.4802 - accuracy: 0.4845 - val_loss: 1.1786 - val_accuracy: 0.5843
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 1.1493 - accuracy: 0.6036 - val_loss: 1.0478 - val_accuracy: 0.6387
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 1.0473 - accuracy: 0.6445 - val_loss: 1.0044 - val_accuracy: 0.6552
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.9929 - accuracy: 0.6631 - val_loss: 0.9717 - val_accuracy: 0.6715
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.9441 - accuracy: 0.6841 - val_loss: 0.9771 - val_accuracy: 0.6724
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.9156 - accuracy: 0.6914 - val_loss: 0.9513 - val_accuracy: 0.6828
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.8964 - accuracy: 0.6975 - val_loss: 0.9880 - val_accuracy: 0.6674
Epoch 

_________________________________________________________________
dense_1 (Dense)              (None, 17)                527       
_________________________________________________________________
dropout_1 (Dropout)          (None, 17)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                180       
Total params: 17,563
Trainable params: 17,563
Non-trainable params: 0
_________________________________________________________________
After pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1

In [16]:
model_test_loss, model_test_acc = model.evaluate(test_images,  test_labels, verbose=2)
step_factor_model_test_loss, step_factor_model_test_acc = step_factor_model.evaluate(test_images,  test_labels, verbose=2)

print(f"Model accuracy before pruning: {model_test_acc*100:.2f}%")
print(f"Model accuracy after pruning: {step_factor_model_test_acc*100:.2f}%")

313/313 - 1s - loss: 1.6454 - accuracy: 0.3963
313/313 - 1s - loss: 1.0017 - accuracy: 0.6610
Model accuracy before pruning: 39.63%
Model accuracy after pruning: 66.10%


In [17]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {step_factor_model.count_params()}")
print(f"Pruned model contains only {(step_factor_model.count_params()/model.count_params())*100:.2f}% of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 12545
Pruned model contains only 16.69% of the original number of parameters.


# Prune a model stepwise to a maximum accuracy loss

In this case, we again want to have only a 3% loss of accuracy. For this there is a second function called `stepwise_accuracy_pruning`. It is started with a pruning factor (for dense and conv) of 5. The weights and filters are removed and the network is re-trained. When the minimum accuracy is reached, the pruning factor is increased. In this case, the previous model is further reduced by the new pruning factor (as is also the case with `stepwise_factor_pruning`). The model is then trained again. This is done until the minimum accuracy is no longer reached.

In [18]:
step_acc_model = pruning.stepwise_accuracy_pruning(model, train_images, train_labels, test_images,
                            test_labels, pruning_acc=None, max_acc_loss=3,
                            prun_factor_dense=5, prun_factor_conv=5, 
                            metric='L1', comp=comp, label_one_hot=False)

313/313 [==============================] - 1s 2ms/step - loss: 1.6454 - accuracy: 0.3963
Start model accuracy: 39.63 %
Minimum required model accuracy: 36.63 %
prun_factor_dense: 5 %
prun_factor_conv: 5 %
pruning step: 1
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928   

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 1.4785 - accuracy: 0.4589 - val_loss: 1.2792 - val_accuracy: 0.5449
pruning step: 3
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 58)        15718     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 58)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 58)          30334     
____________________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.4175 - accuracy: 0.4851 - val_loss: 1.2131 - val_accuracy: 0.5597
pruning step: 4
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 27)        756       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 27)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 53)        12932     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 53)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 53)          25334     
____________________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.3614 - accuracy: 0.5137 - val_loss: 1.1611 - val_accuracy: 0.5909
pruning step: 6
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 23)        644       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 23)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 44)        9152      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 44)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 44)          17468     
____________________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.3174 - accuracy: 0.5357 - val_loss: 1.1670 - val_accuracy: 0.5889
pruning step: 8
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 19)        532       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 19)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 36)        6192      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 36)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 36)          11700     
____________________________________________________________

...

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.5849 - accuracy: 0.4230 - val_loss: 1.3073 - val_accuracy: 0.5448
pruning step: 12
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 11)        308       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 11)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 19)        1900      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 19)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 19)          3268      
___________________________________________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.7000 - accuracy: 0.3635 - val_loss: 1.4322 - val_accuracy: 0.5095
pruning_factor_mean: 25.0 %
weighted_model_params: 4549.375
pruned_model.count_params(): 4681
Increased pruning factors
prun_factor_dense: 30 %
prun_factor_conv: 30 %
pruning step: 12
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 11)        308       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 11)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 19)        1900      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 19)          0         
________________________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8400 - accuracy: 0.3077 - val_loss: 1.5730 - val_accuracy: 0.4362
pruning_factor_mean: 35.0 %
weighted_model_params: 3457.5249999999996
pruned_model.count_params(): 3598
Increased pruning factors
prun_factor_dense: 40 %
prun_factor_conv: 40 %
pruning step: 12
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 11)        308       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 11)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 19)        1900      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 19)          0         
______________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 1.9035 - accuracy: 0.2696 - val_loss: 1.6485 - val_accuracy: 0.3863
pruning_factor_mean: 45.0 %
weighted_model_params: 2365.6749999999997
pruned_model.count_params(): 2659
Increased pruning factors
prun_factor_dense: 50 %
prun_factor_conv: 50 %
pruning step: 12
Finish with pruning
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 11)        308       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 11)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 19)        1900      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 19)          0         
______________

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 2.0104 - accuracy: 0.2216 - val_loss: 1.8080 - val_accuracy: 0.3000
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 1.9008 - accuracy: 0.2520 - val_loss: 1.7555 - val_accuracy: 0.3089
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8649 - accuracy: 0.2638 - val_loss: 1.7256 - val_accuracy: 0.3263
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8427 - accuracy: 0.2721 - val_loss: 1.7478 - val_accuracy: 0.3145
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8282 - accuracy: 0.2779 - val_loss: 1.6962 - val_accuracy: 0.3258
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8108 - accuracy: 0.2846 - val_loss: 1.6705 - val_accuracy: 0.3421
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 1.8076 - accuracy: 0.2865 - val_loss: 1.6669 - val_accuracy: 0.3506
Epoch 

In [19]:
step_acc_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 11)        308       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 11)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 19)        1900      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 19)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 19)          3268      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 19)          0         
_________________________________________________________________
flatten (Flatten)            (None, 76)                0

In [20]:
model_test_loss, model_test_acc = model.evaluate(test_images,  test_labels, verbose=2)
step_acc_model_test_loss, step_acc_model_test_acc = step_acc_model.evaluate(test_images,  test_labels, verbose=2)

print(f"Model accuracy before pruning: {model_test_acc*100:.2f}%")
print(f"Model accuracy after pruning: {step_acc_model_test_acc*100:.2f}%")

313/313 - 1s - loss: 1.6454 - accuracy: 0.3963
313/313 - 1s - loss: 1.3073 - accuracy: 0.5448
Model accuracy before pruning: 39.63%
Model accuracy after pruning: 54.48%


In [21]:
print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {step_acc_model.count_params()}")
print(f"Pruned model contains only {(step_acc_model.count_params()/model.count_params())*100:.2f}% of the original number of parameters.")

Total number of parameters before pruning: 75178
Total number of parameters after pruning: 7279
Pruned model contains only 9.68% of the original number of parameters.
